# Group Assignment
## Covid-19 Data - ETL Process
**Group Participants:**
- Uxía Lojo
- Emiliano Puertas
- María Camila Sanabria
- Joshua Vanderspuy
- Sebastian Zambrano

# **Instructions**

You have to build a big table containing all info from the original CSV files, aggregated by week and country. In other words, every single entry in this macrotable must contain information refered to a specific country in a specific week (inside the range of dates provided by the input files). You can accomplish this goal by aggregating and joining the content of the input tables. Some tips:

Before starting doing join/aggregation operations, elaborate a brief plan about the sequence of operation you will perfom to obtain the macrotable.
Analyze every table separatedly to spot inconsistencies or issues to be solved before the joining/aggregation process.
Granularity of input data is not homogeneous (some table are indexed by time, daily, while others not). Remember that the final table must be indexed by week and country
In this process, you can (and should!) crearte derived variables from existing ones to enrich the data
During the process, a lot of missing values can arise (due to data transformations, due to joining operations, etc.). Handle all this missing values. The resulting table must not contain any.
Do the aggregations at the end of the process. In order to preserve information, first join all data (with the highest granularity), and then do the aggregations to weeks and countries.
You can take the "epidemiology" table as reference to get the range of dates


### Importing Pandas

In [1]:
import pandas as pd

### Reading tables
We will have the macrotable for reference, so it will be downloaded as well

In [2]:
macrotable=pd.read_csv("data/macrotable/macrotable_c")
macrotable.head()

,week,country_name,new_confirmed,new_deceased,new_deceased_confirmed_ratio,population,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older,life_expectancy,new_hospitalized_patients,new_persons_fully_vaccinated
0,2019-12-30/2020-01-05,Germany,1.0,0.0,0.0,82786787.0,7.539514e+06,7.725134e+06,9.713905e+06,1.046693e+07,1.039496e+07,1.350341e+07,1.034477e+07,7.684878e+06,5.413285e+06,NaN,NaN,NaN
1,2020-01-13/2020-01-19,Germany,1.0,0.0,0.0,82786787.0,7.539514e+06,7.725134e+06,9.713905e+06,1.046693e+07,1.039496e+07,1.350341e+07,1.034477e+07,7.684878e+06,5.413285e+06,NaN,NaN,NaN
2,2020-01-20/2020-01-26,Germany,2.0,0.0,0.0,82786787.0,7.539514e+06,7.725134e+06,9.713905e+06,1.046693e+07,1.039496e+07,1.350341e+07,1.034477e+07,7.684878e+06,5.413285e+06,NaN,NaN,NaN
3,2020-01-20/2020-01-26,United States of America,0.0,0.0,0.0,341338766.0,4.218540e+07,4.375102e+07,4.787080e+07,4.560237e+07,4.260209e+07,4.537366e+07,3.840401e+07,2.250539e+07,1.301387e+07,77.871999,2544.6,0.0
4,2020-01-27/2020-02-02,Germany,10.0,0.0,0.0,82786787.0,7.539514e+06,7.725134e+06,9.713905e+06,1.046693e+07,1.039496e+07,1.350341e+07,1.034477e+07,7.684878e+06,5.413285e+06,NaN,NaN,NaN


In [3]:
macrotable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   week                          504 non-null    object 
 1   country_name                  504 non-null    object 
 2   new_confirmed                 504 non-null    float64
 3   new_deceased                  504 non-null    float64
 4   new_deceased_confirmed_ratio  504 non-null    float64
 5   population                    504 non-null    float64
 6   population_age_00_09          504 non-null    float64
 7   population_age_10_19          504 non-null    float64
 8   population_age_20_29          504 non-null    float64
 9   population_age_30_39          504 non-null    float64
 10  population_age_40_49          504 non-null    float64
 11  population_age_50_59          504 non-null    float64
 12  population_age_60_69          504 non-null    float64
 13  popul

In [4]:
macrotable.describe()

,new_confirmed,new_deceased,new_deceased_confirmed_ratio,population,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older,life_expectancy,new_hospitalized_patients,new_persons_fully_vaccinated
count,5.040000e+02,504.000000,504.000000,5.040000e+02,5.040000e+02,5.040000e+02,5.040000e+02,5.040000e+02,5.040000e+02,5.040000e+02,5.040000e+02,5.040000e+02,5.040000e+02,1.350000e+02,135.000000,1.350000e+02
mean,2.465923e+05,2286.992063,0.004206,1.293220e+08,1.464182e+07,1.534543e+07,1.700617e+07,1.693672e+07,1.684963e+07,1.819974e+07,1.492381e+07,9.584126e+06,6.019731e+06,7.787200e+01,2544.600000,1.021703e+06
std,5.406483e+05,4633.637370,0.014912,1.301284e+08,1.678827e+07,1.730140e+07,1.885862e+07,1.754758e+07,1.577286e+07,1.682029e+07,1.447775e+07,8.099128e+06,4.497196e+06,1.569016e-13,3682.569298,2.132879e+06
min,-6.700000e+01,0.000000,0.000000,2.109846e+07,1.915634e+06,2.266531e+06,2.337994e+06,2.942982e+06,3.758505e+06,3.351295e+06,2.417174e+06,1.717090e+06,1.152301e+06,7.787200e+01,4.000000,-1.682610e+05
25%,1.632925e+04,0.000000,0.000000,2.109846e+07,1.915634e+06,2.266531e+06,2.337994e+06,2.942982e+06,3.758505e+06,3.351295e+06,2.417174e+06,1.717090e+06,1.152301e+06,7.787200e+01,616.000000,0.000000e+00
50%,7.102250e+04,31.000000,0.000000,5.544310e+07,4.633566e+06,5.231560e+06,5.623413e+06,6.471216e+06,8.507701e+06,8.610615e+06,6.764526e+06,5.533564e+06,4.066940e+06,7.787200e+01,1336.000000,4.375900e+04
75%,2.507910e+05,1934.500000,0.000715,3.413388e+08,4.218540e+07,4.375102e+07,4.787080e+07,4.560237e+07,4.260209e+07,4.537366e+07,3.840401e+07,2.250539e+07,1.301387e+07,7.787200e+01,2591.300000,1.259466e+06
max,5.656738e+06,25873.000000,0.250000,3.413388e+08,4.218540e+07,4.375102e+07,4.787080e+07,4.560237e+07,4.260209e+07,4.537366e+07,3.840401e+07,2.250539e+07,1.301387e+07,7.787200e+01,23486.000000,1.676596e+07


In [5]:
demographics=pd.read_csv("data/demographics")
demographics.head()

,location_key,population,population_male,population_female,population_rural,population_urban,population_largest_city,population_clustered,population_density,human_development_index,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older
0,DE_BB_12051,72124.0,35617.0,36507.0,NaN,NaN,NaN,NaN,367.4,NaN,6029.0,5183.0,6646.0,9776.0,7690.0,11604.0,10152.0,8681.0,6363.0
1,DE_BB_12052,100219.0,49201.0,51018.0,NaN,NaN,NaN,NaN,609.9,NaN,8542.0,7657.0,10979.0,13671.0,10652.0,15833.0,14258.0,10758.0,7869.0
2,DE_BB_12053,57873.0,28023.0,29850.0,NaN,NaN,NaN,NaN,403.2,NaN,4652.0,4702.0,5977.0,7066.0,6405.0,9325.0,9042.0,6049.0,4655.0
3,DE_BB_12054,178089.0,86179.0,91910.0,NaN,NaN,NaN,NaN,1034.5,NaN,18893.0,15636.0,21671.0,28980.0,23226.0,24768.0,18998.0,14762.0,11155.0
4,DE_BB_12060,182760.0,90615.0,92145.0,NaN,NaN,NaN,NaN,126.8,NaN,16693.0,15671.0,12966.0,23038.0,22973.0,33665.0,27041.0,18553.0,12160.0


In [6]:
demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5097 entries, 0 to 5096
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   location_key                 5097 non-null   object 
 1   population                   5097 non-null   float64
 2   population_male              3743 non-null   float64
 3   population_female            3743 non-null   float64
 4   population_rural             0 non-null      float64
 5   population_urban             0 non-null      float64
 6   population_largest_city      0 non-null      float64
 7   population_clustered         0 non-null      float64
 8   population_density           507 non-null    float64
 9   human_development_index      0 non-null      float64
 10  population_age_00_09         3743 non-null   float64
 11  population_age_10_19         3743 non-null   float64
 12  population_age_20_29         3743 non-null   float64
 13  population_age_30_

In [7]:
epidemiology=pd.read_csv("data/epidemiology")
epidemiology.head()

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
0,2020-03-15,DE_BB_12051,2.0,0.0,2.0,NaN,2.0,0.0,2.0,NaN
1,2020-03-17,DE_BB_12051,1.0,0.0,1.0,NaN,3.0,0.0,3.0,NaN
2,2020-03-19,DE_BB_12051,2.0,0.0,2.0,NaN,5.0,0.0,5.0,NaN
3,2020-03-20,DE_BB_12051,1.0,0.0,1.0,NaN,6.0,0.0,6.0,NaN
4,2020-03-22,DE_BB_12051,2.0,0.0,2.0,NaN,8.0,0.0,8.0,NaN


In [8]:
epidemiology.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3161033 entries, 0 to 3161032
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   date                  object 
 1   location_key          object 
 2   new_confirmed         float64
 3   new_deceased          float64
 4   new_recovered         float64
 5   new_tested            float64
 6   cumulative_confirmed  float64
 7   cumulative_deceased   float64
 8   cumulative_recovered  float64
 9   cumulative_tested     float64
dtypes: float64(8), object(2)
memory usage: 241.2+ MB


In [9]:
health=pd.read_csv("data/health")
health.head()

,location_key,life_expectancy,smoking_prevalence,diabetes_prevalence,infant_mortality_rate,adult_male_mortality_rate,adult_female_mortality_rate,pollution_mortality_rate,comorbidity_mortality_rate,hospital_beds_per_1000,nurses_per_1000,physicians_per_1000,health_expenditure_usd,out_of_pocket_health_expenditure_usd
0,US_AK_02013,86.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US_AK_02016,77.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US_AK_02020,78.034694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,US_AK_02050,78.733333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,US_AK_02060,79.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3022 entries, 0 to 3021
Data columns (total 14 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   location_key                          3022 non-null   object 
 1   life_expectancy                       3022 non-null   float64
 2   smoking_prevalence                    0 non-null      float64
 3   diabetes_prevalence                   0 non-null      float64
 4   infant_mortality_rate                 0 non-null      float64
 5   adult_male_mortality_rate             0 non-null      float64
 6   adult_female_mortality_rate           0 non-null      float64
 7   pollution_mortality_rate              0 non-null      float64
 8   comorbidity_mortality_rate            0 non-null      float64
 9   hospital_beds_per_1000                0 non-null      float64
 10  nurses_per_1000                       0 non-null      float64
 11  physicians_per_10

In [11]:
hospitalizations=pd.read_csv("data/hospitalizations")
hospitalizations.head()

,date,location_key,new_hospitalized_patients,cumulative_hospitalized_patients,current_hospitalized_patients,new_intensive_care_patients,cumulative_intensive_care_patients,current_intensive_care_patients,new_ventilator_patients,cumulative_ventilator_patients,current_ventilator_patients
0,2020-03-23,US_CA_SFO,NaN,NaN,101.0,NaN,NaN,21.0,NaN,NaN,NaN
1,2020-03-24,US_CA_SFO,NaN,NaN,96.0,NaN,NaN,15.0,NaN,NaN,NaN
2,2020-03-25,US_CA_SFO,NaN,NaN,62.0,NaN,NaN,12.0,NaN,NaN,NaN
3,2020-03-26,US_CA_SFO,NaN,NaN,67.0,NaN,NaN,13.0,NaN,NaN,NaN
4,2020-03-27,US_CA_SFO,NaN,NaN,69.0,NaN,NaN,10.0,NaN,NaN,NaN


In [12]:
hospitalizations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6297 entries, 0 to 6296
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   date                                6297 non-null   object 
 1   location_key                        6297 non-null   object 
 2   new_hospitalized_patients           5418 non-null   float64
 3   cumulative_hospitalized_patients    5418 non-null   float64
 4   current_hospitalized_patients       879 non-null    float64
 5   new_intensive_care_patients         0 non-null      float64
 6   cumulative_intensive_care_patients  0 non-null      float64
 7   current_intensive_care_patients     879 non-null    float64
 8   new_ventilator_patients             0 non-null      float64
 9   cumulative_ventilator_patients      0 non-null      float64
 10  current_ventilator_patients         0 non-null      float64
dtypes: float64(9), object(2)
memory usage: 541.

In [13]:
index=pd.read_csv("data/index")
index.head()

,location_key,place_id,wikidata_id,datacommons_id,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,locality_code,locality_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level
0,DE_BB_12051,ChIJN8I30-XAqEcRhUxEEOyL_kg,Q3931,NaN,DE,Germany,BB,Brandenburg,12051,Brandenburg an der Havel,NaN,NaN,DE,DEU,2
1,DE_BB_12052,ChIJX0qVWUJ0CEcROq1_4LUv1FA,Q3214,NaN,DE,Germany,BB,Brandenburg,12052,Cottbus,NaN,NaN,DE,DEU,2
2,DE_BB_12053,ChIJb_u1AiqYB0cRwDteW0YgIQQ,Q4024,NaN,DE,Germany,BB,Brandenburg,12053,Frankfurt an der Oder,NaN,NaN,DE,DEU,2
3,DE_BB_12054,ChIJt9Y6hM31qEcRm-yqC5j4ZcU,Q1711,NaN,DE,Germany,BB,Brandenburg,12054,Potsdam,NaN,NaN,DE,DEU,2
4,DE_BB_12060,ChIJuRSkBF66qUcRCDglm8hflWE,Q6115,NaN,DE,Germany,BB,Brandenburg,12060,Barnim,NaN,NaN,DE,DEU,2


In [14]:
index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5121 entries, 0 to 5120
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   location_key        5121 non-null   object
 1   place_id            5080 non-null   object
 2   wikidata_id         5098 non-null   object
 3   datacommons_id      3329 non-null   object
 4   country_code        5121 non-null   object
 5   country_name        5121 non-null   object
 6   subregion1_code     5121 non-null   object
 7   subregion1_name     5121 non-null   object
 8   subregion2_code     5109 non-null   object
 9   subregion2_name     5109 non-null   object
 10  locality_code       12 non-null     object
 11  locality_name       12 non-null     object
 12  iso_3166_1_alpha_2  5121 non-null   object
 13  iso_3166_1_alpha_3  5121 non-null   object
 14  aggregation_level   5121 non-null   int64 
dtypes: int64(1), object(14)
memory usage: 600.2+ KB


In [15]:
vaccinations=pd.read_csv("data/vaccinations")
vaccinations.head()

,date,location_key,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered,new_persons_vaccinated_pfizer,cumulative_persons_vaccinated_pfizer,...,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,new_persons_vaccinated_sinovac,total_persons_vaccinated_sinovac,new_persons_fully_vaccinated_sinovac,total_persons_fully_vaccinated_sinovac,new_vaccine_doses_administered_sinovac,total_vaccine_doses_administered_sinovac
0,2021-04-26,US_AK_02013,NaN,NaN,NaN,918.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-27,US_AK_02013,NaN,NaN,0.0,918.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-28,US_AK_02013,NaN,NaN,1.0,919.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-29,US_AK_02013,NaN,NaN,0.0,919.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-04-30,US_AK_02013,NaN,NaN,1.0,920.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
vaccinations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562414 entries, 0 to 1562413
Data columns (total 32 columns):
 #   Column                                         Non-Null Count    Dtype  
---  ------                                         --------------    -----  
 0   date                                           1562414 non-null  object 
 1   location_key                                   1562414 non-null  object 
 2   new_persons_vaccinated                         0 non-null        float64
 3   cumulative_persons_vaccinated                  0 non-null        float64
 4   new_persons_fully_vaccinated                   1559194 non-null  float64
 5   cumulative_persons_fully_vaccinated            1562414 non-null  float64
 6   new_vaccine_doses_administered                 0 non-null        float64
 7   cumulative_vaccine_doses_administered          0 non-null        float64
 8   new_persons_vaccinated_pfizer                  0 non-null        float64
 9   cumulative_persons_vacci

### Data Cleaning
Checking duplicates, data types, and missing values taking into account the sample macrotable

|#|Column |Non-Null Count|Dtype|
|-|------|-------------|------|
|0|week|504 non-null|object|
|1|country_name|504 non-null|object|
|2|new_confirmed|504 non-null|float64|
|3|new_deceased|504 non-null|float64|
|4|new_deceased_confirmed_ratio|504 non-null|float64|
|5|population|504 non-null|float64|
|6|population_age_00_09|504 non-null|float64|
|7|population_age_10_19|504 non-null|float64|
|8|population_age_20_29|504 non-null|float64|
|9|population_age_30_39|504 non-null|float64|
|10|population_age_40_49|504 non-null|float64|
|11|population_age_50_59|504 non-null|float64|
|12|population_age_60_69|504 non-null|float64|
|13|population_age_70_79|504 non-null|float64|
|14|population_age_80_and_older|504 non-null|float64|
|15|life_expectancy|135 non-null|float64|
|16|new_hospitalized_patients|135 non-null|float64|
|17|new_persons_fully_vaccinated|135 non-null|float64|

#### Duplicates
Droping duplicates, in case there's any.

In [17]:
epidemiology=epidemiology.drop_duplicates()

In [18]:
health=health.drop_duplicates()

In [19]:
hospitalizations=hospitalizations.drop_duplicates()

In [20]:
index=index.drop_duplicates()

In [21]:
vaccinations=vaccinations.drop_duplicates()

### Checking for empty rows
... and deleting if there is any

In [22]:
epidemiology[epidemiology.isnull().all(axis=1)].shape

(0, 10)

In [23]:
epidemiology=epidemiology.dropna(how='all')

In [24]:
health[health.isnull().all(axis=1)].shape

(0, 14)

In [25]:
health=health.dropna(how='all')

In [26]:
hospitalizations[hospitalizations.isnull().all(axis=1)].shape

(0, 11)

In [27]:
hospitalizations=hospitalizations.dropna(how='all')

In [28]:
index[index.isnull().all(axis=1)].shape

(0, 15)

In [29]:
index=index.dropna(how='all')

In [30]:
vaccinations[vaccinations.isnull().all(axis=1)].shape

(0, 32)

In [31]:
vaccinations=vaccinations.dropna(how='all')

## **Dropping columns**
We will drop columns that are repetitive or not relevant or that have more than 50% of values missing, since they might not be providing enough valuable information for our analysis.

### Demographics table
First, we check for the missing value ratios

In [32]:
miss_demo_ratio=round(demographics.isna().sum()/demographics.shape[0]*100,2)
miss_demo_ratio

location_key                     0.00
population                       0.00
population_male                 26.56
population_female               26.56
population_rural               100.00
population_urban               100.00
population_largest_city        100.00
population_clustered           100.00
population_density              90.05
human_development_index        100.00
population_age_00_09            26.56
population_age_10_19            26.56
population_age_20_29            26.56
population_age_30_39            26.56
population_age_40_49            26.56
population_age_50_59            26.56
population_age_60_69            26.56
population_age_70_79            26.56
population_age_80_and_older     26.56
dtype: float64

We are only keeping location_key, population_female, population_male, population ages 0 until 80 and older

In [33]:
demographics=demographics[['location_key','population','population_male','population_female','population_age_00_09','population_age_10_19','population_age_20_29','population_age_30_39','population_age_40_49','population_age_50_59','population_age_60_69','population_age_70_79','population_age_80_and_older']]

In [34]:
demographics

,location_key,population,population_male,population_female,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older
0,DE_BB_12051,72124.0,35617.0,36507.0,6029.0,5183.0,6646.0,9776.0,7690.0,11604.0,10152.0,8681.0,6363.0
1,DE_BB_12052,100219.0,49201.0,51018.0,8542.0,7657.0,10979.0,13671.0,10652.0,15833.0,14258.0,10758.0,7869.0
2,DE_BB_12053,57873.0,28023.0,29850.0,4652.0,4702.0,5977.0,7066.0,6405.0,9325.0,9042.0,6049.0,4655.0
3,DE_BB_12054,178089.0,86179.0,91910.0,18893.0,15636.0,21671.0,28980.0,23226.0,24768.0,18998.0,14762.0,11155.0
4,DE_BB_12060,182760.0,90615.0,92145.0,16693.0,15671.0,12966.0,23038.0,22973.0,33665.0,27041.0,18553.0,12160.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5092,US_WY_56037,43464.0,22438.0,21026.0,6334.0,6333.0,5488.0,6734.0,5219.0,5534.0,4815.0,2063.0,944.0
5093,US_WY_56039,23384.0,12133.0,11251.0,2461.0,2245.0,3184.0,4184.0,3404.0,2968.0,2855.0,1435.0,648.0
5094,US_WY_56041,20431.0,10339.0,10092.0,3282.0,3182.0,2179.0,2755.0,2349.0,2567.0,2496.0,1116.0,505.0
5095,US_WY_56043,8010.0,4055.0,3955.0,913.0,1162.0,705.0,938.0,902.0,1101.0,1106.0,752.0,431.0


### Epidemiology table
First, we check for the missing value ratios

In [35]:
miss_epid_ratio=round(epidemiology.isna().sum()/epidemiology.shape[0]*100,2)
miss_epid_ratio

date                     0.00
location_key             0.00
new_confirmed            0.11
new_deceased            13.63
new_recovered           90.55
new_tested              98.27
cumulative_confirmed     0.00
cumulative_deceased     13.53
cumulative_recovered    90.55
cumulative_tested       98.27
dtype: float64

Columns new_recovered, new_tested, cumulative_recovered and cumulative_tested have more than 90% of values missing, so we will drop them. We will remove the cumulative columns because they are repetitive for all of our process and we can recalculate later.

In [36]:
epidemiology=epidemiology[['date','location_key','new_confirmed','new_deceased']]
epidemiology.head()

,date,location_key,new_confirmed,new_deceased
0,2020-03-15,DE_BB_12051,2.0,0.0
1,2020-03-17,DE_BB_12051,1.0,0.0
2,2020-03-19,DE_BB_12051,2.0,0.0
3,2020-03-20,DE_BB_12051,1.0,0.0
4,2020-03-22,DE_BB_12051,2.0,0.0


In [37]:
epidemiology

,date,location_key,new_confirmed,new_deceased
0,2020-03-15,DE_BB_12051,2.0,0.0
1,2020-03-17,DE_BB_12051,1.0,0.0
2,2020-03-19,DE_BB_12051,2.0,0.0
3,2020-03-20,DE_BB_12051,1.0,0.0
4,2020-03-22,DE_BB_12051,2.0,0.0
...,...,...,...,...
3161028,2022-05-09,US_WY_56045,0.0,0.0
3161029,2022-05-10,US_WY_56045,-1.0,0.0
3161030,2022-05-11,US_WY_56045,0.0,0.0
3161031,2022-05-12,US_WY_56045,0.0,0.0


### Health Table
First, we check for the missing value ratios

In [38]:
miss_health_ratio=round(health.isna().sum()/health.shape[0]*100,2)
miss_health_ratio

location_key                              0.0
life_expectancy                           0.0
smoking_prevalence                      100.0
diabetes_prevalence                     100.0
infant_mortality_rate                   100.0
adult_male_mortality_rate               100.0
adult_female_mortality_rate             100.0
pollution_mortality_rate                100.0
comorbidity_mortality_rate              100.0
hospital_beds_per_1000                  100.0
nurses_per_1000                         100.0
physicians_per_1000                     100.0
health_expenditure_usd                  100.0
out_of_pocket_health_expenditure_usd    100.0
dtype: float64

Only Location_key and Life Expectancy have enough information, so we are keeping only both of these columns

In [39]:
health=health[['location_key','life_expectancy']]
health.head()

,location_key,life_expectancy
0,US_AK_02013,86.900000
1,US_AK_02016,77.750000
2,US_AK_02020,78.034694
3,US_AK_02050,78.733333
4,US_AK_02060,79.900000


In [40]:
health

,location_key,life_expectancy
0,US_AK_02013,86.900000
1,US_AK_02016,77.750000
2,US_AK_02020,78.034694
3,US_AK_02050,78.733333
4,US_AK_02060,79.900000
...,...,...
3017,US_WY_56037,79.177778
3018,US_WY_56039,81.950000
3019,US_WY_56041,77.700000
3020,US_WY_56043,80.600000


### Hospitalizations Table
First, we check the missing values ratio

In [41]:
miss_hosp_ratio=round(hospitalizations.isna().sum()/hospitalizations.shape[0]*100,2)
miss_hosp_ratio

date                                    0.00
location_key                            0.00
new_hospitalized_patients              13.96
cumulative_hospitalized_patients       13.96
current_hospitalized_patients          86.04
new_intensive_care_patients           100.00
cumulative_intensive_care_patients    100.00
current_intensive_care_patients        86.04
new_ventilator_patients               100.00
cumulative_ventilator_patients        100.00
current_ventilator_patients           100.00
dtype: float64

Given the rules we established for dropping a column, we will only keep the following ones for this table: Date, location_key, new_hospitalized_patients, and cumulative_hospitalized_patients.

In [42]:
hospitalizations=hospitalizations[['date','location_key','new_hospitalized_patients']]
hospitalizations

,date,location_key,new_hospitalized_patients
0,2020-03-23,US_CA_SFO,NaN
1,2020-03-24,US_CA_SFO,NaN
2,2020-03-25,US_CA_SFO,NaN
3,2020-03-26,US_CA_SFO,NaN
4,2020-03-27,US_CA_SFO,NaN
...,...,...,...
6292,2022-08-15,US_NY_NYC,107.0
6293,2022-08-16,US_NY_NYC,98.0
6294,2022-08-17,US_NY_NYC,75.0
6295,2022-08-18,US_NY_NYC,41.0


In [43]:
hospitalizations

,date,location_key,new_hospitalized_patients
0,2020-03-23,US_CA_SFO,NaN
1,2020-03-24,US_CA_SFO,NaN
2,2020-03-25,US_CA_SFO,NaN
3,2020-03-26,US_CA_SFO,NaN
4,2020-03-27,US_CA_SFO,NaN
...,...,...,...
6292,2022-08-15,US_NY_NYC,107.0
6293,2022-08-16,US_NY_NYC,98.0
6294,2022-08-17,US_NY_NYC,75.0
6295,2022-08-18,US_NY_NYC,41.0


### Index
First, we check for missing values

In [44]:
miss_index_ratio=round(index.isna().sum()/index.shape[0]*100,2)
miss_index_ratio

location_key           0.00
place_id               0.80
wikidata_id            0.45
datacommons_id        34.99
country_code           0.00
country_name           0.00
subregion1_code        0.00
subregion1_name        0.00
subregion2_code        0.23
subregion2_name        0.23
locality_code         99.77
locality_name         99.77
iso_3166_1_alpha_2     0.00
iso_3166_1_alpha_3     0.00
aggregation_level      0.00
dtype: float64

For this table, locality_code and locality_name have 99% of values missing, so we are dropping these columns. Additionaly, we are dropping the place_id (we already have a location_key), the wikidata_id, datacommons_id, ISO and aggregation_level (it's not relevant for the analysis), We are also dropping the subregions because we only want country for our analysis

In [45]:
index=index[['location_key','country_code','country_name']]
index

,location_key,country_code,country_name
0,DE_BB_12051,DE,Germany
1,DE_BB_12052,DE,Germany
2,DE_BB_12053,DE,Germany
3,DE_BB_12054,DE,Germany
4,DE_BB_12060,DE,Germany
...,...,...,...
5116,US_WY_56037,US,United States of America
5117,US_WY_56039,US,United States of America
5118,US_WY_56041,US,United States of America
5119,US_WY_56043,US,United States of America


In [46]:
index

,location_key,country_code,country_name
0,DE_BB_12051,DE,Germany
1,DE_BB_12052,DE,Germany
2,DE_BB_12053,DE,Germany
3,DE_BB_12054,DE,Germany
4,DE_BB_12060,DE,Germany
...,...,...,...
5116,US_WY_56037,US,United States of America
5117,US_WY_56039,US,United States of America
5118,US_WY_56041,US,United States of America
5119,US_WY_56043,US,United States of America


### Vaccinations table
First, we check for missing values ratio

In [47]:
miss_vacc_ratio=round(vaccinations.isna().sum()/vaccinations.shape[0]*100,2)
miss_vacc_ratio

date                                               0.00
location_key                                       0.00
new_persons_vaccinated                           100.00
cumulative_persons_vaccinated                    100.00
new_persons_fully_vaccinated                       0.21
cumulative_persons_fully_vaccinated                0.00
new_vaccine_doses_administered                   100.00
cumulative_vaccine_doses_administered            100.00
new_persons_vaccinated_pfizer                    100.00
cumulative_persons_vaccinated_pfizer             100.00
new_persons_fully_vaccinated_pfizer              100.00
cumulative_persons_fully_vaccinated_pfizer       100.00
new_vaccine_doses_administered_pfizer            100.00
cumulative_vaccine_doses_administered_pfizer     100.00
new_persons_vaccinated_moderna                   100.00
cumulative_persons_vaccinated_moderna            100.00
new_persons_fully_vaccinated_moderna             100.00
cumulative_persons_fully_vaccinated_moderna     

For this table, we are only keeping columns date, location_key, and new_persons_fully_vaccinated.

In [48]:
vaccinations=vaccinations[['date','location_key','new_persons_fully_vaccinated']]
vaccinations

,date,location_key,new_persons_fully_vaccinated
0,2021-04-26,US_AK_02013,NaN
1,2021-04-27,US_AK_02013,0.0
2,2021-04-28,US_AK_02013,1.0
3,2021-04-29,US_AK_02013,0.0
4,2021-04-30,US_AK_02013,1.0
...,...,...,...
1562409,2022-08-19,US_WY_56045,0.0
1562410,2022-08-20,US_WY_56045,0.0
1562411,2022-08-21,US_WY_56045,0.0
1562412,2022-08-22,US_WY_56045,0.0


## Checking Data Types
First, we check the data types of the macrotable for reference

In [49]:
macrotable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   week                          504 non-null    object 
 1   country_name                  504 non-null    object 
 2   new_confirmed                 504 non-null    float64
 3   new_deceased                  504 non-null    float64
 4   new_deceased_confirmed_ratio  504 non-null    float64
 5   population                    504 non-null    float64
 6   population_age_00_09          504 non-null    float64
 7   population_age_10_19          504 non-null    float64
 8   population_age_20_29          504 non-null    float64
 9   population_age_30_39          504 non-null    float64
 10  population_age_40_49          504 non-null    float64
 11  population_age_50_59          504 non-null    float64
 12  population_age_60_69          504 non-null    float64
 13  popul

#### Epidemiology Table

In [50]:
epidemiology.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3161033 entries, 0 to 3161032
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   date           object 
 1   location_key   object 
 2   new_confirmed  float64
 3   new_deceased   float64
dtypes: float64(2), object(2)
memory usage: 96.5+ MB


Turn dates into date format

In [51]:
epidemiology.date=pd.to_datetime(epidemiology.date)
epidemiology.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3161033 entries, 0 to 3161032
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   date           datetime64[ns]
 1   location_key   object        
 2   new_confirmed  float64       
 3   new_deceased   float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 96.5+ MB


### Hospitalizations table

In [52]:
#Converting date from object into date format
hospitalizations.date=pd.to_datetime(hospitalizations.date)
hospitalizations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6297 entries, 0 to 6296
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       6297 non-null   datetime64[ns]
 1   location_key               6297 non-null   object        
 2   new_hospitalized_patients  5418 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 147.7+ KB


### Vaccinations table

In [53]:
vaccinations.date=pd.to_datetime(vaccinations.date)
vaccinations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562414 entries, 0 to 1562413
Data columns (total 3 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   date                          1562414 non-null  datetime64[ns]
 1   location_key                  1562414 non-null  object        
 2   new_persons_fully_vaccinated  1559194 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 35.8+ MB


In [54]:
epidemiology.location_key.value_counts()

location_key
US_NY_NYC      943
US_CA_06059    937
US_CA_06037    936
US_CA_06085    931
US_CA_06011    930
              ... 
ES_MD_28117     35
ES_MD_28069     34
ES_CT_43041     29
ES_MD_28016     20
ES_MD_28001     20
Name: count, Length: 4802, dtype: int64

In [55]:
epidemiology[epidemiology.new_confirmed.isnull()]

,date,location_key,new_confirmed,new_deceased
532060,2021-11-16,ES_MD_28001,NaN,NaN
532080,2020-04-03,ES_MD_28002,NaN,NaN
532259,2021-02-09,ES_MD_28003,NaN,NaN
532317,2020-03-16,ES_MD_28004,NaN,NaN
532514,2020-03-03,ES_MD_28005,NaN,NaN
...,...,...,...,...
3157198,2020-03-24,US_WY_56037,NaN,NaN
3157979,2020-03-18,US_WY_56039,NaN,NaN
3158766,2020-04-02,US_WY_56041,NaN,NaN
3159538,2020-03-27,US_WY_56043,NaN,NaN


In [56]:
epidemiology[(epidemiology['location_key']=='ES_MD_28001')]

,date,location_key,new_confirmed,new_deceased
532060,2021-11-16,ES_MD_28001,NaN,NaN
532061,2021-11-23,ES_MD_28001,1.0,NaN
532062,2021-11-30,ES_MD_28001,2.0,NaN
532063,2021-12-07,ES_MD_28001,-1.0,NaN
532064,2021-12-14,ES_MD_28001,1.0,NaN
532065,2021-12-21,ES_MD_28001,0.0,NaN
532066,2021-12-28,ES_MD_28001,0.0,NaN
532067,2022-01-04,ES_MD_28001,0.0,NaN
532068,2022-01-11,ES_MD_28001,1.0,NaN
532069,2022-01-18,ES_MD_28001,2.0,NaN


In [57]:
epidemiology[(epidemiology.date=='2021-11-16') & (epidemiology['location_key']=='ES_MD_28001')]

,date,location_key,new_confirmed,new_deceased
532060,2021-11-16,ES_MD_28001,NaN,NaN


### Create the Weeks column
We will create a function to make the process more effective

In [58]:
def week_dates(dt):
    st_date=(dt-pd.Timedelta(days=dt.weekday())).date()
    end_date=(dt+pd.Timedelta(days=6-dt.weekday())).date()
    return f"{st_date}/{end_date}"

In [59]:
def week_column(df: pd.DataFrame):
    df["week"] = df["date"].apply(week_dates)
    return df

In [60]:
epidemiology=week_column(epidemiology)

In [61]:
hospitalizations=week_column(hospitalizations)

In [62]:
hospitalizations.head()

,date,location_key,new_hospitalized_patients,week
0,2020-03-23,US_CA_SFO,NaN,2020-03-23/2020-03-29
1,2020-03-24,US_CA_SFO,NaN,2020-03-23/2020-03-29
2,2020-03-25,US_CA_SFO,NaN,2020-03-23/2020-03-29
3,2020-03-26,US_CA_SFO,NaN,2020-03-23/2020-03-29
4,2020-03-27,US_CA_SFO,NaN,2020-03-23/2020-03-29


In [63]:
vaccinations=week_column(vaccinations)
vaccinations.head()

,date,location_key,new_persons_fully_vaccinated,week
0,2021-04-26,US_AK_02013,NaN,2021-04-26/2021-05-02
1,2021-04-27,US_AK_02013,0.0,2021-04-26/2021-05-02
2,2021-04-28,US_AK_02013,1.0,2021-04-26/2021-05-02
3,2021-04-29,US_AK_02013,0.0,2021-04-26/2021-05-02
4,2021-04-30,US_AK_02013,1.0,2021-04-26/2021-05-02


### **Exploring our data more in depth**
We will check how many countries we have in our informational tables

In [64]:
index.country_code.value_counts()

country_code
US    3228
ES    1378
DE     412
IT     103
Name: count, dtype: int64

In [65]:
health.location_key.value_counts()

location_key
US_AK_02013    1
US_OH_39023    1
US_OH_39005    1
US_OH_39007    1
US_OH_39009    1
              ..
US_KY_21045    1
US_KY_21047    1
US_KY_21049    1
US_KY_21051    1
US_WY_56045    1
Name: count, Length: 3022, dtype: int64

Health table only has info for the US.

In [66]:
health.location_key.str[:2].value_counts()

location_key
US    3022
Name: count, dtype: int64

In [67]:
epidemiology.location_key.value_counts()

location_key
US_NY_NYC      943
US_CA_06059    937
US_CA_06037    936
US_CA_06085    931
US_CA_06011    930
              ... 
ES_MD_28117     35
ES_MD_28069     34
ES_CT_43041     29
ES_MD_28016     20
ES_MD_28001     20
Name: count, Length: 4802, dtype: int64

Epidemiology has information for the 4 countries

In [68]:
epidemiology.location_key.str[:2].value_counts()

location_key
US    2501894
ES     331747
DE     233559
IT      93833
Name: count, dtype: int64

In [69]:
hospitalizations.location_key.value_counts()

location_key
US_NY_36005    903
US_NY_36047    903
US_NY_36061    903
US_NY_36081    903
US_NY_36085    903
US_NY_NYC      903
US_CA_SFO      879
Name: count, dtype: int64

Hospitalizations only has information for the US.

In [70]:
hospitalizations.location_key.str[:2].value_counts()

location_key
US    6297
Name: count, dtype: int64

In [71]:
vaccinations.location_key.value_counts()

location_key
US_AK_02013    485
US_OK_40039    485
US_OK_40019    485
US_OK_40021    485
US_OK_40023    485
              ... 
US_KY_21165    485
US_KY_21167    485
US_KY_21169    485
US_KY_21171    485
US_AK_02261    229
Name: count, Length: 3222, dtype: int64

Vaccinations only have information for one country

In [72]:
vaccinations.location_key.str[:2].value_counts()

location_key
US    1562414
Name: count, dtype: int64

In [73]:
demographics.describe()

,population,population_male,population_female,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older
count,5.097000e+03,3.743000e+03,3.743000e+03,3.743000e+03,3.743000e+03,3.743000e+03,3.743000e+03,3.743000e+03,3.743000e+03,3743.000000,3743.000000,3743.000000
mean,9.822780e+04,6.270148e+04,6.482959e+04,1.443720e+04,1.507825e+04,1.678602e+04,1.663845e+04,1.641535e+04,1.794909e+04,14751.427197,9495.748063,5979.535667
std,3.213663e+05,1.778261e+05,1.863752e+05,4.423678e+04,4.385307e+04,5.316021e+04,5.283404e+04,4.900389e+04,4.852718e+04,37887.495446,23815.958069,15566.848542
min,2.200000e+01,4.100000e+01,4.500000e+01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+01,6.000000e+00,1.500000e+01,16.000000,7.000000,0.000000
25%,4.872000e+03,6.235500e+03,6.166500e+03,1.447500e+03,1.542000e+03,1.405000e+03,1.408500e+03,1.461500e+03,1.743000e+03,1674.000000,1044.000000,603.000000
50%,2.009200e+04,1.661900e+04,1.670100e+04,3.911000e+03,4.146000e+03,4.025000e+03,3.868000e+03,3.895000e+03,4.600000e+03,4258.000000,2621.000000,1463.000000
75%,7.281600e+04,5.583050e+04,5.691250e+04,1.207450e+04,1.301350e+04,1.395950e+04,1.366250e+04,1.356750e+04,1.620900e+04,13951.000000,8564.000000,5087.500000
max,1.010371e+07,4.980981e+06,5.122730e+06,1.228873e+06,1.252274e+06,1.576001e+06,1.484454e+06,1.362194e+06,1.309380e+06,992699.000000,548702.000000,349134.000000


In [ ]:
health.describe()

,life_expectancy
count,3022.000000
mean,77.871999
std,7.732088
min,69.050000
25%,75.863542
50%,77.636250
75%,79.466667
max,401.307595


: 